In [1]:
import numpy as np
import os
import sys
import pandas as pd

import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

import pylab as pl
import h5py

import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['CUDA_VISIBLE_DEVICES']="0" 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


from sklearn.metrics import confusion_matrix

base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3.0'
os.chdir(base_dir)

from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.HelperFunctions import HelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.TimeAugmentor import TimeAugmentor
from Classes.DataProcessing.NoiseAugmentor import NoiseAugmentor
from Classes.DataProcessing.RamLoader import RamLoader
from Classes.DataProcessing.RamGenerator import RamGenerator
from Classes.Modeling.InceptionTimeModel import InceptionTimeModel
from Classes.Modeling.NarrowSearchRam import NarrowSearchRam
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Modeling.ResultFitter import ResultFitter
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
from Classes.Modeling.GridSearchResultProcessor import GridSearchResultProcessor
import json
#from Classes import Tf_shutup
#Tf_shutup.Tf_shutup()

from livelossplot import PlotLossesKeras



from matplotlib.colors import ListedColormap

plt.rcParams["figure.figsize"]= (15,15)
helper = HelperFunctions()

import sys
ISCOLAB = 'google.colab' in sys.modules

import random
import pprint



1 Physical GPUs, 1 Logical GPUs


In [2]:
load_args = {
    'earth_explo_only' : False,
    'noise_earth_only' : False,
    'noise_not_noise' : True,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 1,
    'seed' : 1,
    'subsample_size' : 0.4,
    'balance_non_train_set' : True,
    'use_true_test_set' : False,
    'even_balance' : True
}
loadData = LoadData(**load_args)
full_ds, train_ds, val_ds, test_ds = loadData.get_datasets()
noise_ds = loadData.noise_ds
handler = DataHandler(loadData)

if load_args['earth_explo_only']:
    full_and_noise_ds = np.concatenate((full_ds, noise_ds))
    timeAug = TimeAugmentor(handler, full_and_noise_ds, seed = load_args['seed'])
else:
    timeAug = TimeAugmentor(handler, full_ds, seed = load_args['seed'])

2 3
{'noise': 105999, 'earthquake': 105999, 'explosion': 102808}


In [3]:
########### Preprocessing ###########
use_noise_augmentor = True
use_time_augmentor = True
detrend = False
use_scaler = True
use_highpass = False
highpass_freq = 0.2

use_tensorboard = True
use_livelossplot = False
use_custom = False
use_reduce_lr = True

In [4]:

class ResultModelGetter:

    def __init__(self, loadData, handler, helper, timeAug = None, scaler = None, noiseAug = None):
        self.loadData = loadData
        self.handler = handler
        self.helper = helper
        self.timeAug = timeAug
        self.scaler = scaler
        self.noiseAug = noiseAug

    def model_from_result(self, result_name, input_shape, num_classes = 2, by_index = False, index = 0, top_index = 0, sort_by = ['val_f1', 'train_f1'], early_stop = False):
        df = self.load_df(result_name, num_classes)
        df_f1 = df.copy()
        df_f1 = self.add_f1(df_f1)
        df_f1.convert_dtypes().dtypes

        if by_index:
            model_df = df_f1.iloc[index]
        else:
            ascending = False
            if sort_by == ['val_loss', 'train_loss']:
                ascending = True
            df_f1 = df_f1.sort_values(by=sort_by, axis = 0, ascending = ascending).reset_index()
            model_df = df_f1.iloc[top_index]

        model_dict = self.row_to_dict(model_df)
        batch_size = model_dict['batch_size']
        epochs = model_dict['epochs']
        learning_rate = model_dict['learning_rate']
        num_channels = model_dict['num_channels']
        fit_args = {"batch_size" : batch_size, "epochs" : epochs, "learning_rate" : learning_rate, "num_channels" : num_channels}

        del model_dict['batch_size']
        del model_dict['epochs']
        del model_dict['learning_rate']
        del model_dict['num_channels']
        
        return self.build_model_by_name(result_name, model_dict, input_shape, num_classes, fit_args)

    def row_to_dict(self, model_df):
        keys = list(model_df.keys())
        # Assumes 10 columns dedicated to results and the rest to hyperparams
        hyper_keys = keys[:len(keys) - 10]
        model_dict = model_df[:len(hyper_keys)].to_dict()
        del model_dict['index']
        return model_dict

    def build_model_by_name(self, result_name, model_dict, input_shape, num_classes, fit_args):
        model_name = result_name.split('_')[1]
        if model_name == 'InceptionTime':
            return self.build_incep_model(model_dict, input_shape, num_classes, fit_args)
        else:
            raise Exception(f"Not implemented {model_name} yet")

    def build_incep_model(self, model_dict, input_shape, num_classes, fit_args):
        optimizer = self.helper.get_optimizer(model_dict['optimizer'], fit_args["learning_rate"])
        compile_args = self.helper.generate_model_compile_args(optimizer, num_classes)
        input = {"input_shape" : input_shape, "nr_classes" : num_classes}
        build_dict = {**input, **model_dict}
        build_dict["optimizer"] = optimizer

        inceptionTime = InceptionTimeModel(**build_dict)
        model = inceptionTime.build_model(input_shape, num_classes)
        print(model.summary())
        return model, fit_args

        
    def load_df(self, result_name, num_classes):
        return GridSearchResultProcessor().get_results_df_by_name(result_name, num_classes)

    def add_f1(self, df):
        df_f1 = df
        df_f1.columns=df_f1.columns.str.strip()
        all_train_precision = df_f1['train_precision']
        all_train_recall = df_f1['train_recall']
        all_val_precision = df_f1['val_precision']
        all_val_recall = df_f1['val_recall']
        f1_train = self.create_f1_list(all_train_precision, all_train_recall)
        f1_val = self.create_f1_list(all_val_precision, all_val_recall)
        df_f1['train_f1'] = f1_train
        df_f1['val_f1'] = f1_val
        return df_f1

    def f1_score(self, precision, recall):
        f1 = 2*((precision*recall)/(precision + recall))
        return f1

    def create_f1_list(self, precision_df, recall_df):
        f1 = []
        for i in range(len(precision_df)):
            f1.append(self.f1_score(precision_df.loc[i], recall_df.loc[i]))
        return f1


In [5]:
resultModel = ResultModelGetter(loadData, handler, HelperFunctions(), None, None, None)
model, fit_args = resultModel.model_from_result('results_InceptionTime_noiseNotNoise_timeAug_sscale_noiseAug_earlyS.csv', (3, 6000), by_index = False, index = 10, top_index = 0, sort_by = ['val_accuracy', 'train_accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3, 6000)]    0                                            
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 3, 6000)      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 3, 32)        7680000     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 3, 32)        3840000     input_1[0][0]                    
______________________________________________________________________________________________

In [6]:
########### Preprocessing ###########
use_noise_augmentor = True
use_time_augmentor = True
detrend = False
use_scaler = True
use_highpass = False
highpass_freq = 0.2

use_tensorboard = True
use_livelossplot = False
use_custom = False
use_reduce_lr = True

scaler = None
noiseAug = None
if use_time_augmentor:
    timeAug.fit()
if use_scaler:
    scaler = StandardScalerFitter(train_ds, timeAug).fit_scaler(detrend = detrend)
if use_noise_augmentor:
    noiseAug = NoiseAugmentor(train_ds, use_scaler, scaler, loadData, timeAug)
num_ds, channels, timesteps = handler.get_trace_shape_no_cast(train_ds, use_time_augmentor)
input_shape = (channels, timesteps)

Fit process completed after 305.30140590667725 seconds. Total datapoints fitted: 84160.
Average time per datapoint: 0.003627630773605956


In [7]:
ramLoader = RamLoader(handler, timeAug, scaler)
x_train, y_train = ramLoader.load_to_ram(train_ds, False)
x_val, y_val = ramLoader.load_to_ram(val_ds, False)

Starting loading to RAM
Completed loading to RAM
Starting loading to RAM
Completed loading to RAM


In [8]:
callbacks = []

def clear_tensorboard_dir():
    import os
    import shutil
    path = f"{base_dir}/Tensorboard_dir/fit"
    files = os.listdir(path)
    print(files)
    for f in files:
        shutil.rmtree(os.path.join(path,f))

if use_tensorboard:
    import datetime
    clear_tensorboard_dir()
    #%tensorboard --logdir tensorboard_dir/fit
    log_dir = f"{base_dir}/tensorboard_dir/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    callbacks.append(tensorboard_callback)
    # tensorboard --logdir='/media/tord/T7/Thesis_ssd/MasterThesis3.0/tensorboard_dir/fit'

if use_custom:
    custom_callback = CustomCallback(data_gen)
    callbacks.append(custom_callback)
elif use_livelossplot:
    callbacks.append(PlotLossesKeras())
elif use_reduce_lr:
    callbacks.append(tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50,
                                                          min_lr=0.0001))

['20210210-020130', '20210210-020427', '20210210-025257', '20210210-025530', '20210210-025806', '20210210-030202', '20210210-031148', '20210210-031221', '20210210-031640', '20210210-032037', '20210210-032056', '20210210-032449', '20210210-032533', '20210210-032621', '20210210-041554', '20210210-042047', '20210210-042409', '20210210-044355', '20210210-044949', '20210210-045022', '20210210-045442', '20210210-045741', '20210210-050010', '20210210-050948', '20210210-051022', '20210210-051320', '20210210-051622', '20210210-051918', '20210210-052334', '20210210-052604', '20210210-052623', '20210210-053237', '20210210-053733', '20210210-054127', '20210210-054357', '20210210-054749', '20210210-055211', '20210210-055436', '20210210-064324', '20210210-064558', '20210210-064953', '20210210-071709', '20210210-071940', '20210210-072232', '20210210-072502', '20210210-072602', '20210210-072832', '20210210-080711']


In [9]:



gen = RamGenerator(loadData, handler)

batch_size = fit_args["batch_size"]

train_gen = gen.data_generator(x_train, y_train, batch_size)
val_gen = gen.data_generator(x_val, y_val, batch_size)

args = {'steps_per_epoch' : helper.get_steps_per_epoch(train_ds, batch_size),
        'epochs' : 100,
        'validation_data' : val_gen,
        'validation_steps' : helper.get_steps_per_epoch(val_ds, batch_size),
        'verbose' : 1,
        'use_multiprocessing' : False, 
        'workers' : 1,
        'callbacks' : callbacks
}

model_fit = model.fit(train_gen, **args)

Epoch 1/100
123/123 [==============================] - 8s 34ms/step - loss: 0.6632 - precision_1: 0.5999 - binary_accuracy: 0.5892 - recall_1: 0.5030 - val_loss: 0.6711 - val_precision_1: 0.9882 - val_binary_accuracy: 0.5632 - val_recall_1: 0.1231
Epoch 2/100
123/123 [==============================] - 4s 30ms/step - loss: 0.5613 - precision_1: 0.8165 - binary_accuracy: 0.7263 - recall_1: 0.5816 - val_loss: 1.1909 - val_precision_1: 0.4966 - val_binary_accuracy: 0.4960 - val_recall_1: 0.9957
Epoch 3/100
123/123 [==============================] - 4s 30ms/step - loss: 0.5205 - precision_1: 0.8155 - binary_accuracy: 0.7534 - recall_1: 0.6524 - val_loss: 0.5569 - val_precision_1: 0.9825 - val_binary_accuracy: 0.7089 - val_recall_1: 0.4221
Epoch 4/100
123/123 [==============================] - 4s 31ms/step - loss: 0.4960 - precision_1: 0.8327 - binary_accuracy: 0.7708 - recall_1: 0.6762 - val_loss: 0.5069 - val_precision_1: 0.8552 - val_binary_accuracy: 0.8158 - val_recall_1: 0.7579
Epoch 5/

Epoch 34/100
123/123 [==============================] - 4s 31ms/step - loss: 0.3387 - precision_1: 0.9076 - binary_accuracy: 0.8582 - recall_1: 0.7956 - val_loss: 2.6690 - val_precision_1: 0.4978 - val_binary_accuracy: 0.4984 - val_recall_1: 0.9912
Epoch 35/100
123/123 [==============================] - 4s 31ms/step - loss: 0.3324 - precision_1: 0.9093 - binary_accuracy: 0.8594 - recall_1: 0.7961 - val_loss: 2.3869 - val_precision_1: 0.4983 - val_binary_accuracy: 0.4995 - val_recall_1: 0.9881
Epoch 36/100
123/123 [==============================] - 4s 32ms/step - loss: 0.3270 - precision_1: 0.9181 - binary_accuracy: 0.8655 - recall_1: 0.8006 - val_loss: 7.8001 - val_precision_1: 0.4971 - val_binary_accuracy: 0.4971 - val_recall_1: 0.9971
Epoch 37/100
123/123 [==============================] - 4s 33ms/step - loss: 0.3262 - precision_1: 0.9123 - binary_accuracy: 0.8655 - recall_1: 0.8062 - val_loss: 3.6603 - val_precision_1: 0.4972 - val_binary_accuracy: 0.4972 - val_recall_1: 0.9979
Epoc

Epoch 67/100
123/123 [==============================] - 4s 31ms/step - loss: 0.2550 - precision_1: 0.9332 - binary_accuracy: 0.8957 - recall_1: 0.8491 - val_loss: 4.5277 - val_precision_1: 0.4925 - val_binary_accuracy: 0.4880 - val_recall_1: 0.9769
Epoch 68/100
123/123 [==============================] - 4s 32ms/step - loss: 0.2490 - precision_1: 0.9341 - binary_accuracy: 0.8986 - recall_1: 0.8540 - val_loss: 4.4132 - val_precision_1: 0.4977 - val_binary_accuracy: 0.4982 - val_recall_1: 0.9956
Epoch 69/100
123/123 [==============================] - 4s 32ms/step - loss: 0.2435 - precision_1: 0.9394 - binary_accuracy: 0.9021 - recall_1: 0.8558 - val_loss: 0.7442 - val_precision_1: 0.6758 - val_binary_accuracy: 0.6720 - val_recall_1: 0.6540
Epoch 70/100
123/123 [==============================] - 4s 31ms/step - loss: 0.2431 - precision_1: 0.9378 - binary_accuracy: 0.9026 - recall_1: 0.8589 - val_loss: 3.5336 - val_precision_1: 0.4995 - val_binary_accuracy: 0.5017 - val_recall_1: 0.9881
Epoc

Epoch 100/100
123/123 [==============================] - 4s 32ms/step - loss: 0.1799 - precision_1: 0.9546 - binary_accuracy: 0.9289 - recall_1: 0.8987 - val_loss: 1.9640 - val_precision_1: 0.5174 - val_binary_accuracy: 0.5348 - val_recall_1: 0.9573
